In [ ]:
#Create Tree sets with 100 raxml-ng runs with interim trees, removing duplicates etc.

In [ ]:
# %load LuiseUtil.py
import os
import shutil
import math
from ete3 import Tree
from Bio import Phylo
import matplotlib
import matplotlib.pyplot as plt
from codecs import decode
import struct

from Bio import Phylo, AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.AlignIO.PhylipIO import PhylipWriter


raxml_ng_path = './../../tools/raxml-ng/build/bin/raxml-ng'
standard_raxml_path = './../../tools/standard-RAxML-master/raxmlHPC-AVX '

tree_dir = 'data/trees/'
alignment_dir = 'data/language_alignments/'
sitelh_dir = 'data/siteLH/'
drawings_dir = 'output/drawings/'
weight_calibration_dir = 'data/weight_calibration/'
site_congruence_dir = 'data/site_congruence/'
lh_dir = 'data/lh/'
indices_dir = 'data/indices/'


#tree_space_name = 'space.trees'
geo_tree_name = "geo_duration.tree"
cognate_tree_name = "cognate_ie_compatible.tree"
cognate_ml_tree_name = "cognate_ml.tree"

morpho_alignment_name = "morpho.phy"



def read_trees_from_ete(tree_set_names):
    trees = []
    for tree_set in tree_set_names:
        l_file = open(tree_dir + tree_set, 'r')
        lines = l_file.readlines()
        for line in lines:
            trees.append(Tree(line))
    return trees

#def read_tree_space_ete():
#    return read_trees_from_ete([tree_space_name])


def eliminate_topological_duplicates_ete(tree_set_names, out_file_name):
    unique_list = []
    tree_set_ete = read_trees_from_ete(tree_set_names)
    i = 0
    for t1 in tree_set_ete:
        unique = True
        for t2 in unique_list:
            rf = rf_distance_ete(t1, t2)
            if rf == 0:
                unique = False
                break
        if unique:
            unique_list.append(t1)
        i = i + 1
    file_name = tree_dir + out_file_name
    with open(file_name, 'w+') as tree_file:
        for tree in unique_list:
            tree_file.write(tree.write()+"\n")
    print(file_name + " created")

def set_neg_branches_zero_ete(tree):
    for node in tree.traverse("postorder"):
        if node.dist < 0:
            node.dist = 0
    return tree

def set_neg_branches_zero(tree_set_name):
    trees = read_trees_from_ete([tree_set_name])
    for tree in trees:
        tree = set_neg_branches_zero_ete(tree)
    file_name = tree_dir + (".".join(tree_set_name.split(".")[:-1])) + ".nonneg.trees"
    with open(file_name, 'w+') as tree_file:
        for tree in trees:
            tree_file.write(tree.write()+"\n")



#def create_tree_space_from(tree_set_names):
#    tree_space_ete = read_trees_from_ete(tree_set_names)
#    file_name = tree_dir + tree_space_name
#    with open(file_name, 'w+') as tree_file:
#        for tree in tree_space_ete:
#            tree_file.write(tree.write()+"\n")
#    print(str(len(tree_space_ete)) + " trees written to " + file_name)

def read_geo_tree_ete():
    return Tree(tree_dir + geo_tree_name)

def read_cognate_tree_ete():
    return Tree(tree_dir + cognate_tree_name)

def rf_distance_ete(t1, t2):
    rf, max_rf, common_leaves, parts_t1, parts_t2,discard_t1, discart_t2 = t1.robinson_foulds(t2, unrooted_trees = True)
    if max_rf == 0:
        print("?!")
        return 0
    return rf/max_rf

def rf_distances_ete(ref_tree, tree_set):
    distances = []
    for tree in tree_set:
        distances.append(rf_distance_ete(ref_tree, tree))
    return distances


def calculate_rf_distances_raxml(ref_tree_name, tree_set_names):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    dir_string = tree_dir + ref_tree_name
    for tree_set in tree_set_names:
        dir_string = dir_string + tree_dir + tree_set
    os.system("cat " + dir_string + " > temp/all.trees")
    os.system(raxml_ng_path + " --rfdist --tree temp/all.trees --prefix temp/foo > temp/bar.txt")
    l_file = open('temp/foo.raxml.rfDistances', 'r')
    lines = l_file.readlines()
    i = 0
    line = lines[i].split("\t")
    distances = []
    while(line[0] == '0'):
        distances.append(float(line[3]))
        i+=1
        line = lines[i].split("\t")
    shutil.rmtree("temp/", ignore_errors=True)
    return distances


def evaluate_lh_raxml(tree_name, alignment_name, optimize = True):
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system(raxml_ng_path + ' --evaluate --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix foo --nofiles' +
              optimize_string + '> out.txt')
    l_file = open('out.txt', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    os.remove("out.txt")
    return lh







def calculate_site_lh_raxml_ete(tree_ete, alignment_name, optimize= True):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    tree_ete.write(outfile="temp/foo.tree")
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system(raxml_ng_path + ' --sitelh --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree temp/foo.tree --prefix temp/foo ' +
              optimize_string + '> temp/bar.txt')
    l_file = open('temp/foo.raxml.log', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    with open('temp/foo.raxml.siteLH' , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    #siteLH = [float(data[i]) for i in range(5, len(data))]
    siteLH = [data[i] for i in range(5, len(data))]
    shutil.rmtree("temp/", ignore_errors=True)
    return [lh, siteLH]




def print_tree_with_phylo(tree_name, save = False):
    tree = Phylo.read(tree_dir + tree_name, "newick")
    tree.ladderize()
    fig = plt.figure(figsize=(10, 10), dpi=100)
    axes = fig.add_subplot(1, 1, 1)
    axes.set_title(tree_name)
    Phylo.draw(tree, axes=axes, do_show=False)
    if save:
        plt.savefig(drawings_dir + tree_name + '.png', dpi=fig.dpi)


def fix_beast_output(tree_set_name):
    beast_file = open(tree_dir + tree_set_name, 'r')
    lines = beast_file.readlines()
    i = 0
    while not lines[i].startswith("\tTranslate"):
        i = i+1
    translate = []
    while not lines[i].startswith(";"):
        if lines[i].endswith(",\n"):
            translate.append(lines[i].split(" ")[-1][:-2])
        else:
            translate.append(lines[i].split(" ")[-1][:-1])
        i=i+1
    i = i+1
    with open(tree_dir + rm_end(tree_set_name) + "_fixed.trees" , 'w+') as fixed_file:
        for j in range(i, len(lines)-1):
            tree = Tree(lines[j].split(" ")[-1])
            for leaf in tree.iter_leaves():
                leaf.name = translate[int(leaf.name)]
            fixed_file.write(tree.write() + "\n")

def rm_end(file_name):
    return '.'.join(file_name.split('.') [:-1])

def lh_file_name(tree_set_name, alignment_name, optimize):
    optimize_string = "_opt-branches="
    if optimize:
        optimize_string = optimize_string + "on"
    else:
        optimize_string = optimize_string + "off"
    return lh_dir + rm_end(alignment_name) + '_' + rm_end(tree_set_name)  + optimize_string + '.lh'

def lh_raw_file_name(tree_set_name, alignment_name, optimize):
    optimize_string = "_opt-branches="
    if optimize:
        optimize_string = optimize_string + "on"
    else:
        optimize_string = optimize_string + "off"
    return lh_dir + rm_end(alignment_name) + '_' + rm_end(tree_set_name)  + optimize_string + '_raw.lh'


def site_lh_file_name(tree_name, alignment_name, optimize):
    optimize_string = "_opt-branches="
    if optimize:
        optimize_string = optimize_string + "on"
    else:
        optimize_string = optimize_string + "off"
    return sitelh_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + optimize_string + '.raxml.siteLH'

def site_lh_raw_file_name(tree_name, alignment_name, optimize):
    optimize_string = "_opt-branches="
    if optimize:
        optimize_string = optimize_string + "on"
    else:
        optimize_string = optimize_string + "off"
    return sitelh_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + optimize_string + '_raw.raxml.siteLH'

def weight_calibration_file_name(tree_name, alignment_name):
    return weight_calibration_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + '.raxml.weightCalibration'

def site_congruence_file_name(tree_name, alignment_name):
    return site_congruence_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + '.raxml.siteCongruence'

def optimized_tree_file_name(tree_name, alignment_name):
    return tree_dir + rm_end(tree_name) + '_optimized_' + rm_end(alignment_name)  + '.tree'

def ml_trees_file_name(alignment_name, start_trees = ""):
    if start_trees == "":
        return tree_dir + rm_end(alignment_name) + ".raxml.mlTrees"
    else:
        return tree_dir + rm_end(alignment_name) + "_" + start_trees + ".raxml.mlTrees"

def best_tree_file_name(alignment_name, start_trees = ""):
    if start_trees == "":
        return tree_dir + rm_end(alignment_name) + ".raxml.bestTree"
    else:
        return tree_dir + rm_end(alignment_name) + "_" + start_trees + ".raxml.bestTree"


def read_lhs(tree_set_name, alignment_name, optimize):
    lhs = []
    site_lhs = []
    with open(lh_file_name(tree_set_name, alignment_name, optimize) , 'r') as file:
        lines = file.read().split("\n")
    for line in lines[:-1]:
        line_data = line.split("\t")
        lhs.append(float(line_data[0]))
        site_lhs.append([float(el) for el in line_data[1].split(" ")[:-1]])
    return (lhs, site_lhs)

def read_lhs_raw(tree_set_name, alignment_name, optimize):
    lhs = []
    site_lhs = []
    with open(lh_raw_file_name(tree_set_name, alignment_name, optimize) , 'r') as file:
        lines = file.read().split("\n")
    for line in lines[:-1]:
        line_data = line.split("\t")
        lhs.append(float(line_data[0]))
        site_lhs.append([bin_to_float(el) for el in line_data[1].split(" ")[:-1]])
    return (lhs, site_lhs)

def read_site_lh(tree_name, alignment_name, optimize):
    with open(site_lh_file_name(tree_name, alignment_name, optimize) , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    return [float(data[i]) for i in range(5, len(data))]

def read_site_lh_raw(tree_name, alignment_name, optimize):
    with open(site_lh_raw_file_name(tree_name, alignment_name, optimize) , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    return [bin_to_float(data[i]) for i in range(5, len(data))]

def read_weight_calibration(tree_name, alignment_name):
    with open(weight_calibration_file_name(tree_name, alignment_name) , 'r') as file:
        data = file.read().split(" ")
    return [int(data[i]) for i in range(len(data) - 1)]

def read_site_congruence(tree_name, alignment_name):
    with open(site_congruence_file_name(tree_name, alignment_name) , 'r') as file:
        data = file.read().split("\n")
    return [float(data[i].split(" ")[1]) for i in range(len(data) - 1)]

def read_optimized_tree(tree_name, alignment_name):
    return Tree(optimized_tree_file_name(tree_name, alignment_name))

def read_ml_trees(alignment_name, start_trees = ""):
    tree_file = open(ml_trees_file_name(alignment_name, start_trees))
    return [Tree(line[:-1]) for line in tree_file.readlines()]

def read_best_tree(alignment_name, start_trees = ""):
    tree_file = open(best_tree_file_name(alignment_name, start_trees))
    return Tree(tree_file.readlines()[0][:-1])

def calculate_lhs_raxml(tree_set_name, alignment_name, optimize = False):
    tree_set = read_trees_from_ete([tree_set_name])
    results = [calculate_site_lh_raxml_ete(tree, alignment_name, optimize) for tree in tree_set]
    with open(lh_raw_file_name(tree_set_name, alignment_name, optimize), 'w+') as out_file:
        for result in results:
            out_file.write(str(result[0]))
            out_file.write("\t")
            for site_lh in result[1]:
                #out_file.write(str(site_lh))
                out_file.write(site_lh)
                out_file.write(" ")
            out_file.write("\n")

def calculate_site_lh_raxml(tree_name, alignment_name, optimize= True):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system(raxml_ng_path + ' --sitelh --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix temp/foo ' +
              optimize_string + '> temp/bar.txt')
    l_file = open('temp/foo.raxml.log', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    #os.system("cat temp/foo.raxml.siteLH > " + site_lh_file_name(tree_name, alignment_name, optimize))
    os.system("cat temp/foo.raxml.siteLH > " + site_lh_raw_file_name(tree_name, alignment_name, optimize))
    with open('temp/foo.raxml.siteLH' , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    #siteLH = [float(data[i]) for i in range(5, len(data))]
    siteLH = [data[i] for i in range(5, len(data))]
    shutil.rmtree("temp/", ignore_errors=True)
    return [lh, siteLH]

def calculate_weight_calibration_raxml(tree_name, alignment_name):
    os.system(standard_raxml_path + ' -f u -p 12345 -t ' + tree_dir + tree_name +
              ' -m BINGAMMA -s ' + alignment_dir + alignment_name +
              ' -n calibration > bar.txt')
    os.system('cat RAxML_weights.calibration > '
              + weight_calibration_file_name(tree_name, alignment_name))
    os.remove('bar.txt')
    os.remove('RAxML_weights.calibration')
    os.remove('RAxML_info.calibration')

def calculate_site_congruence_raxml(tree_name, alignment_name):
    os.system(standard_raxml_path + ' -f S -t ' + tree_dir + tree_name +
              ' -m BINGAMMA -s ' + alignment_dir + alignment_name +
              ' -n congruence > bar.txt')
    os.system('cat RAxML_SiteSpecificPlacementBias.congruence > '
              + site_congruence_file_name(tree_name, alignment_name))
    os.remove('bar.txt')
    os.remove('RAxML_SiteSpecificPlacementBias.congruence')
    os.remove('RAxML_info.congruence')

def calculate_optimized_tree_raxml(tree_name, alignment_name):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    os.system(raxml_ng_path + ' --evaluate --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix temp/foo ' + '> out.txt')
    os.system('cat temp/foo.raxml.bestTree > ' + tree_dir + rm_end(tree_name) + "_optimized_" + rm_end(alignment_name) + '.tree')
    shutil.rmtree("temp/", ignore_errors=True)


def calculate_ml_trees_raxml(alignment_name, start_trees = ""):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    if start_trees == "":
        os.system(raxml_ng_path + ' --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --seed 2 --model BIN+G --prefix temp/foo > temp/bar.txt')
    else:
        os.system(raxml_ng_path + ' --msa ' + alignment_dir + alignment_name +
            ' --tree ' + start_trees +
            ' --threads 2 --seed 2 --model BIN+G --prefix temp/foo > temp/bar.txt')
    os.system('cat temp/foo.raxml.mlTrees > ' + ml_trees_file_name(alignment_name, start_trees))
    os.system('cat temp/foo.raxml.bestTree > ' + best_tree_file_name(alignment_name, start_trees))



def get_site_lh(tree_name, alignment_name, optimize):
    if not os.path.isfile(site_lh_file_name(tree_name, alignment_name, optimize)):
        print("Currently not possible, use site_lh_raw")
        #calculate_site_lh_raxml(tree_name, alignment_name, optimize)
    return read_site_lh(tree_name, alignment_name, optimize)

def get_site_lh_raw(tree_name, alignment_name, optimize):
    if not os.path.isfile(site_lh_raw_file_name(tree_name, alignment_name, optimize)):
        calculate_site_lh_raxml(tree_name, alignment_name, optimize)
    return read_site_lh_raw(tree_name, alignment_name, optimize)


def get_weight_calibration(tree_name, alignment_name):
    if not os.path.isfile(weight_calibration_file_name(tree_name, alignment_name)):
        calculate_weight_calibration_raxml(tree_name, alignment_name)
    return read_weight_calibration(tree_name, alignment_name)

def get_site_congruence(tree_name, alignment_name):
    if not os.path.isfile(site_congruence_file_name(tree_name, alignment_name)):
        calculate_site_congruence_raxml(tree_name, alignment_name)
    return read_site_congruence(tree_name, alignment_name)

def get_optimized_tree(tree_name, alignment_name):
    if not os.path.isfile(optimized_tree_file_name(tree_name, alignment_name)):
        calculate_optimized_tree_raxml(tree_name, alignment_name)
    return read_optimized_tree(tree_name, alignment_name)

def get_double_optimized_tree(tree_name, alignment_name):
    if not os.path.isfile(optimized_tree_file_name(tree_name, alignment_name)):
        calculate_optimized_tree_raxml(tree_name, alignment_name)
    optimized_tree_name = optimized_tree_file_name(tree_name, alignment_name).split('/')[-1]
    if not os.path.isfile(optimized_tree_file_name(optimized_tree_name, alignment_name)):
        calculate_optimized_tree_raxml(optimized_tree_name, alignment_name)
    return read_optimized_tree(optimized_tree_name, alignment_name)


def get_optimized_tree(tree_name, alignment_name):
    if not os.path.isfile(optimized_tree_file_name(tree_name, alignment_name)):
        calculate_optimized_tree_raxml(tree_name, alignment_name)
    return read_optimized_tree(tree_name, alignment_name)


def get_lhs(tree_set_name, alignment_name, optimize):
    if not os.path.isfile(lh_file_name(tree_set_name, alignment_name, optimize)):
        #calculate_lhs_raxml(tree_set_name, alignment_name, optimize)
        print("Currently not possible, use lhs_raw")
    return read_lhs(tree_set_name, alignment_name, optimize)

def get_lhs_raw(tree_set_name, alignment_name, optimize):
    if not os.path.isfile(lh_raw_file_name(tree_set_name, alignment_name, optimize)):
        calculate_lhs_raxml(tree_set_name, alignment_name, optimize)
    return read_lhs_raw(tree_set_name, alignment_name, optimize)


def get_ml_trees(alignment_name, start_trees = ""):
    if not os.path.isfile(ml_trees_file_name(alignment_name, start_trees)):
        calculate_ml_trees_raxml(alignment_name, start_trees)
    return read_ml_trees(alignment_name, start_trees)

def get_best_tree(alignment_name, start_trees = ""):
    if not os.path.isfile(best_tree_file_name(alignment_name, start_trees)):
        calculate_ml_trees_raxml(alignment_name, start_trees)
    return read_best_tree(alignment_name, start_trees)





def average_branch_length(tree_set):
    avg = 0
    cnt = 0
    for tree in tree_set:
        for node in tree.traverse():
            avg = avg + node.dist
            cnt = cnt + 1
    avg = avg / cnt
    return avg



def interval_branch_length(tree_set):
    lower = 1
    upper = 0
    for tree in tree_set:
        for node in tree.traverse():
            lower = min(lower, node.dist)
            upper = max(upper, node.dist)
    return (lower, upper)



def variance_branch_length(tree_set):
    avg = average_branch_length(tree_set)
    var = 0
    cnt = 0
    for tree in tree_set:
        for node in tree.traverse():
            diff = node.dist - avg
            var = var + (diff * diff)
            cnt = cnt + 1
    return var / cnt


def bin_to_float(b):
    return struct.unpack('>d', decode('%%0%dx' % (8 << 1) % int(b, 2), 'hex')[-8:])[0]


def rf_dist_matrix(tree_dict):
    for i, name1 in enumerate(tree_dict):
        for j, name2 in enumerate(tree_dict):
            if name2 <= name1:
                continue
            rf_str = str(rf_distance_ete(tree_dict[name1], tree_dict[name2]))
            print("RF distance of " + name1 + " to " + name2 + ": " + rf_str)













In [12]:
def create_tree_sets(alignment_name, prefix):
    os.system("./interim_trees.sh " + alignment_name + " " + prefix)
    eliminate_topological_duplicates_ete([prefix + "pars100rand100bin.raxml.interimTrees"], 
                                         prefix + "pars100rand100bin_unique.raxml.interimTrees")
    eliminate_topological_duplicates_ete([prefix + "pars100rand100bing.raxml.interimTrees"], 
                                         prefix + "pars100rand100bing_unique.raxml.interimTrees")
    #eliminate_topological_duplicates_ete([prefix + "pars100rand100bin.raxml.mlrees"], 
    #                                     prefix + "pars100rand100bin_unique.raxml.mlTrees")
    #eliminate_topological_duplicates_ete([prefix + "pars100rand100bing.raxml.mlTrees"], 
    #                                     prefix + "pars100rand100bing_unique.raxml.mlTrees")
    eliminate_topological_duplicates_ete([prefix + "pars100rand100bin_unique.raxml.interimTrees",
                                          prefix + "pars100rand100bing_unique.raxml.interimTrees"],
                                         prefix + "all_interim.trees")
    eliminate_topological_duplicates_ete([prefix + "pars100rand100bin.raxml.startTree",
                                          prefix + "pars100rand100bing.raxml.startTree"],
                                         prefix + "all_start.trees")
    eliminate_topological_duplicates_ete([prefix + "all_start.trees",
                                          prefix + "all_interim.trees"],
                                         prefix + "all.trees")
    os.remove("data/trees/" + prefix + "pars100rand100bing.raxml.bestModel")
    os.remove("data/trees/" + prefix + "pars100rand100bing.raxml.bestTree")
    os.remove("data/trees/" + prefix + "pars100rand100bing.raxml.log")
    os.remove("data/trees/" + prefix + "pars100rand100bing.raxml.rba")
    os.remove("data/trees/" + prefix + "pars100rand100bin.raxml.bestModel")
    os.remove("data/trees/" + prefix + "pars100rand100bin.raxml.bestTree")
    os.remove("data/trees/" + prefix + "pars100rand100bin.raxml.log")
    os.remove("data/trees/" + prefix + "pars100rand100bin.raxml.rba")
    
create_tree_sets("sim_cognate_ie_compatible.phy", "sim_cog_")


RAxML-NG v. 1.1.0-master released on 04.08.2022 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, 2 cores, 15 GB RAM

RAxML-NG was called at 23-Jan-2023 16:50:16 as follows:

./../tools/raxml-ng/build/bin/raxml-ng --msa data/language_alignments/sim_cognate_ie_compatible.phy --model BIN+G --prefix data/trees/sim_cog_pars100rand100bing --threads 2 --seed 2 --tree pars{100},rand{100}

Analysis options:
  run mode: ML tree search
  start tree(s): random (100) + parsimony (100)
  random seed: 2
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  fast spr radius: AUTO
  spr subtree cutoff: 1.000000
  branch lengths: proportional (ML estimate, al

[00:01:57] [worker #0] ML tree search #89, logLikelihood: -4272.298697
[00:01:57] [worker #1] ML tree search #92, logLikelihood: -4272.298697
[00:02:00] [worker #1] ML tree search #94, logLikelihood: -4272.298697
[00:02:01] [worker #0] ML tree search #91, logLikelihood: -4272.298697
[00:02:03] [worker #1] ML tree search #96, logLikelihood: -4272.298697
[00:02:04] [worker #0] ML tree search #93, logLikelihood: -4272.298697
[00:02:06] [worker #1] ML tree search #98, logLikelihood: -4272.298697
[00:02:07] [worker #0] ML tree search #95, logLikelihood: -4272.298697
[00:02:09] [worker #1] ML tree search #100, logLikelihood: -4272.298697
[00:02:10] [worker #0] ML tree search #97, logLikelihood: -4272.298697
[00:02:11] [worker #1] ML tree search #102, logLikelihood: -4272.298697
[00:02:13] [worker #0] ML tree search #99, logLikelihood: -4272.298697
[00:02:13] [worker #1] ML tree search #104, logLikelihood: -4272.298697
[00:02:15] [worker #0] ML tree search #101, logLikelihood: -4272.298697
[0

[00:00:01] [worker #0] ML tree search #1, logLikelihood: -4272.346492
[00:00:02] [worker #1] ML tree search #2, logLikelihood: -4272.346492
[00:00:03] [worker #1] ML tree search #4, logLikelihood: -4272.346492
[00:00:03] [worker #0] ML tree search #3, logLikelihood: -4272.346492
[00:00:05] [worker #1] ML tree search #6, logLikelihood: -4272.346492
[00:00:05] [worker #0] ML tree search #5, logLikelihood: -4272.346492
[00:00:06] [worker #1] ML tree search #8, logLikelihood: -4272.346492
[00:00:07] [worker #0] ML tree search #7, logLikelihood: -4272.346492
[00:00:08] [worker #0] ML tree search #9, logLikelihood: -4272.346492
[00:00:08] [worker #1] ML tree search #10, logLikelihood: -4272.346492
[00:00:09] [worker #1] ML tree search #12, logLikelihood: -4272.346492
[00:00:10] [worker #0] ML tree search #11, logLikelihood: -4272.346492
[00:00:11] [worker #1] ML tree search #14, logLikelihood: -4272.346492
[00:00:12] [worker #0] ML tree search #13, logLikelihood: -4272.346492
[00:00:12] [wor

[00:01:10] [worker #0] ML tree search #115, logLikelihood: -4272.346492
[00:01:10] [worker #1] ML tree search #122, logLikelihood: -4272.346492
[00:01:11] [worker #0] ML tree search #117, logLikelihood: -4272.346492
[00:01:11] [worker #1] ML tree search #124, logLikelihood: -4272.346492
[00:01:12] [worker #0] ML tree search #119, logLikelihood: -4272.346492
[00:01:12] [worker #1] ML tree search #126, logLikelihood: -4272.346492
[00:01:12] [worker #1] ML tree search #128, logLikelihood: -4272.346492
[00:01:13] [worker #0] ML tree search #121, logLikelihood: -4272.346492
[00:01:13] [worker #1] ML tree search #130, logLikelihood: -4272.346492
[00:01:13] [worker #0] ML tree search #123, logLikelihood: -4272.346492
[00:01:14] [worker #1] ML tree search #132, logLikelihood: -4272.346492
[00:01:14] [worker #0] ML tree search #125, logLikelihood: -4272.346492
[00:01:15] [worker #1] ML tree search #134, logLikelihood: -4272.346492
[00:01:15] [worker #0] ML tree search #127, logLikelihood: -4272

tree 171
tree 172
tree 173
tree 174
tree 175
tree 176
tree 177
tree 178
tree 179
tree 180
tree 181
tree 182
tree 183
tree 184
tree 185
tree 186
tree 187
tree 188
tree 189
tree 190
tree 191
tree 192
tree 193
tree 194
tree 195
tree 196
tree 197
tree 198
tree 199
tree 200
tree 201
tree 202
tree 203
tree 204
tree 205
tree 206
tree 207
tree 208
tree 209
tree 210
tree 211
tree 212
tree 213
tree 214
tree 215
tree 216
tree 217
tree 218
tree 219
tree 220
tree 221
tree 222
tree 223
tree 224
tree 225
tree 226
tree 227
tree 228
tree 229
tree 230
tree 231
tree 232
tree 233
tree 234
tree 235
tree 236
tree 237
tree 238
tree 239
tree 240
tree 241
tree 242
tree 243
tree 244
tree 245
tree 246
tree 247
tree 248
tree 249
tree 250
tree 251
tree 252
tree 253
tree 254
tree 255
tree 256
tree 257
tree 258
tree 259
tree 260
tree 261
tree 262
tree 263
tree 264
tree 265
tree 266
tree 267
tree 268
tree 269
tree 270
tree 271
tree 272
tree 273
tree 274
tree 275
tree 276
tree 277
tree 278
tree 279
tree 280
tree 281
t

...

tree 447
tree 448
tree 449
tree 450
tree 451
tree 452
tree 453
tree 454
tree 455
tree 456
tree 457
tree 458
tree 459
tree 460
tree 461
tree 462
tree 463
tree 464
tree 465
tree 466
tree 467
tree 468
tree 469
tree 470
tree 471
tree 472
tree 473
tree 474
tree 475
tree 476
tree 477
tree 478
tree 479
tree 480
tree 481
tree 482
tree 483
tree 484
tree 485
tree 486
tree 487
tree 488
tree 489
tree 490
tree 491
tree 492
tree 493
tree 494
tree 495
tree 496
tree 497
tree 498
tree 499
tree 500
tree 501
tree 502
tree 503
tree 504
tree 505
tree 506
tree 507
tree 508
tree 509
tree 510
tree 511
tree 512
tree 513
tree 514
tree 515
tree 516
tree 517
tree 518
tree 519
tree 520
tree 521
tree 522
tree 523
tree 524
tree 525
tree 526
tree 527
tree 528
tree 529
tree 530
tree 531
tree 532
tree 533
tree 534
tree 535
tree 536
tree 537
tree 538
tree 539
tree 540
tree 541
tree 542
tree 543
tree 544
tree 545
tree 546
tree 547
tree 548
tree 549
tree 550
tree 551
tree 552
tree 553
tree 554
tree 555
tree 556
tree 557
t

tree 1328
tree 1329
tree 1330
tree 1331
tree 1332
tree 1333
tree 1334
tree 1335
tree 1336
tree 1337
tree 1338
tree 1339
tree 1340
tree 1341
tree 1342
tree 1343
tree 1344
tree 1345
tree 1346
tree 1347
tree 1348
tree 1349
tree 1350
tree 1351
tree 1352
tree 1353
tree 1354
tree 1355
tree 1356
tree 1357
tree 1358
tree 1359
tree 1360
tree 1361
tree 1362
tree 1363
tree 1364
tree 1365
tree 1366
tree 1367
tree 1368
tree 1369
tree 1370
tree 1371
tree 1372
tree 1373
tree 1374
tree 1375
tree 1376
tree 1377
tree 1378
tree 1379
tree 1380
tree 1381
tree 1382
tree 1383
tree 1384
tree 1385
tree 1386
tree 1387
tree 1388
tree 1389
tree 1390
tree 1391
tree 1392
tree 1393
tree 1394
tree 1395
tree 1396
tree 1397
tree 1398
tree 1399
tree 1400
tree 1401
tree 1402
tree 1403
tree 1404
tree 1405
tree 1406
tree 1407
tree 1408
tree 1409
tree 1410
tree 1411
tree 1412
tree 1413
tree 1414
tree 1415
tree 1416
tree 1417
tree 1418
tree 1419
tree 1420
tree 1421
tree 1422
tree 1423
tree 1424
tree 1425
tree 1426
tree 1427


tree 2155
tree 2156
tree 2157
tree 2158
tree 2159
tree 2160
tree 2161
tree 2162
tree 2163
tree 2164
tree 2165
tree 2166
tree 2167
tree 2168
tree 2169
tree 2170
tree 2171
tree 2172
tree 2173
tree 2174
tree 2175
tree 2176
tree 2177
tree 2178
tree 2179
tree 2180
tree 2181
tree 2182
tree 2183
tree 2184
tree 2185
tree 2186
tree 2187
tree 2188
tree 2189
tree 2190
tree 2191
tree 2192
tree 2193
tree 2194
tree 2195
tree 2196
tree 2197
tree 2198
tree 2199
tree 2200
tree 2201
tree 2202
tree 2203
tree 2204
tree 2205
tree 2206
tree 2207
tree 2208
tree 2209
tree 2210
tree 2211
tree 2212
tree 2213
tree 2214
tree 2215
tree 2216
tree 2217
tree 2218
tree 2219
tree 2220
tree 2221
tree 2222
tree 2223
tree 2224
tree 2225
tree 2226
tree 2227
tree 2228
tree 2229
tree 2230
tree 2231
tree 2232
tree 2233
tree 2234
tree 2235
tree 2236
tree 2237
tree 2238
tree 2239
tree 2240
tree 2241
tree 2242
tree 2243
tree 2244
tree 2245
tree 2246
tree 2247
tree 2248
tree 2249
tree 2250
tree 2251
tree 2252
tree 2253
tree 2254


tree 2983
tree 2984
tree 2985
tree 2986
tree 2987
tree 2988
tree 2989
tree 2990
tree 2991
tree 2992
tree 2993
tree 2994
tree 2995
tree 2996
tree 2997
tree 2998
tree 2999
tree 3000
tree 3001
tree 3002
tree 3003
tree 3004
tree 3005
tree 3006
tree 3007
tree 3008
tree 3009
tree 3010
tree 3011
tree 3012
tree 3013
tree 3014
tree 3015
tree 3016
tree 3017
tree 3018
tree 3019
tree 3020
tree 3021
tree 3022
tree 3023
tree 3024
tree 3025
tree 3026
tree 3027
tree 3028
tree 3029
tree 3030
tree 3031
tree 3032
tree 3033
tree 3034
tree 3035
tree 3036
tree 3037
tree 3038
tree 3039
tree 3040
tree 3041
tree 3042
tree 3043
tree 3044
tree 3045
tree 3046
tree 3047
tree 3048
tree 3049
tree 3050
tree 3051
tree 3052
tree 3053
tree 3054
tree 3055
tree 3056
tree 3057
tree 3058
tree 3059
tree 3060
tree 3061
tree 3062
tree 3063
tree 3064
tree 3065
tree 3066
tree 3067
tree 3068
tree 3069
tree 3070
tree 3071
tree 3072
tree 3073
tree 3074
tree 3075
tree 3076
tree 3077
tree 3078
tree 3079
tree 3080
tree 3081
tree 3082


51 unique trees
tree 51
52 unique trees
tree 52
53 unique trees
tree 53
54 unique trees
tree 54
55 unique trees
tree 55
56 unique trees
tree 56
57 unique trees
tree 57
58 unique trees
tree 58
59 unique trees
tree 59
60 unique trees
tree 60
61 unique trees
tree 61
62 unique trees
tree 62
63 unique trees
tree 63
64 unique trees
tree 64
65 unique trees
tree 65
66 unique trees
tree 66
67 unique trees
tree 67
68 unique trees
tree 68
69 unique trees
tree 69
70 unique trees
tree 70
71 unique trees
tree 71
72 unique trees
tree 72
73 unique trees
tree 73
74 unique trees
tree 74
75 unique trees
tree 75
76 unique trees
tree 76
77 unique trees
tree 77
78 unique trees
tree 78
79 unique trees
tree 79
80 unique trees
tree 80
81 unique trees
tree 81
82 unique trees
tree 82
83 unique trees
tree 83
84 unique trees
tree 84
85 unique trees
tree 85
86 unique trees
tree 86
87 unique trees
tree 87
88 unique trees
tree 88
89 unique trees
tree 89
90 unique trees
tree 90
91 unique trees
tree 91
92 unique trees
